In [31]:
import geopandas as gpd
import pandas as pd
import shapely as shp
import seaborn as sns
import numpy as np

import json

import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *

In [33]:
engine = create_engine('postgresql://kyleslugg:@localhost:5432/FoodAccess')

In [43]:
block_centroids = gpd.read_postgis('SELECT * FROM block_centroids', engine)
markets = gpd.read_postgis('SELECT * FROM markets', engine)

### Find Closest Markets to Each Block Centroid

In [25]:
conn = psycopg2.connect("dbname=FoodAccess user=kyleslugg")
cur = conn.cursor()

In [12]:
nearest_three_markets = pd.DataFrame({'nearest_1':'000000', 
                        'nearest_2':'000000', 
                        'nearest_3':'000000'}, index=['00000'])

for geoid in block_centroids['geoid10']:
    query = f'''
        SELECT id 
        FROM markets
        ORDER BY markets.geom <-> (SELECT geom from block_centroids bc2 WHERE bc2.geoid10 = '{geoid}')
        LIMIT 3;'''
    cur.execute(query)
    results = cur.fetchall()
    
    nearest_three_markets = nearest_three_markets.append(pd.DataFrame({'nearest_1':results[0][0],
                       'nearest_2':results[1][0],
                       'nearest_3':results[2][0]}, index=[geoid]))

In [26]:
conn.close()

nearest_three_markets.to_sql('nearest_markets', engine, if_exists='replace')

### Find closest vertex in street_centerlines graph to markets and block centroids

In [27]:
def get_nearest_vertex(identifier, table_name, id_col_name, cursor):
    query = f'''SELECT id FROM street_centerlines_vertices_pgr 
    ORDER BY the_geom <-> (SELECT geom FROM {table_name} t2 WHERE t2.{id_col_name} = '{identifier}' ) LIMIT 1;'''
    cursor.execute(query)
    result = cursor.fetchone()
    return result[0]

In [44]:
conn = psycopg2.connect("dbname=FoodAccess user=kyleslugg")
cur = conn.cursor()

block_centroids['nearest_node'] = block_centroids['geoid10'].apply(lambda x: get_nearest_vertex(x, 'block_centroids', 'geoid10', cur))
markets['nearest_node'] = markets['id'].apply(lambda x: get_nearest_vertex(x, 'markets', 'id', cur))

conn.close()

In [45]:
block_centroids['geom'] = block_centroids['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
block_centroids.to_sql('block_centroids', engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

markets['geom'] = markets['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
markets.to_sql('markets', engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})

In [46]:
block_centroids = gpd.read_postgis('SELECT * FROM block_centroids', engine)
markets = gpd.read_postgis('SELECT * FROM markets', engine)

### Compute walking time from block centroids to three closest markets

This analysis draws on a network graph constructed using pgrouting and a shapefile of NYC street centerlines. Costs (in minutes) have been computed for each street segment on the assumption of a 5km/h walking speed.

In [77]:
def get_ending_nodes(geoid):
        query = f'''SELECT nearest_1, nearest_2, nearest_3 
                    FROM nearest_markets 
                    WHERE index ='{geoid}';'''
        cur.execute(query)
        market_ids = cur.fetchall()[0]
        node_ids = []
        
        for market in market_ids:
            cur.execute(f'''SELECT nearest_node FROM markets WHERE id = '{market}';''')
            node_ids.append(cur.fetchone()[0])
        
        return node_ids

In [78]:
conn = psycopg2.connect("dbname=FoodAccess user=kyleslugg")
cur = conn.cursor()

node_pairs = pd.DataFrame({})
node_pairs['geoid10'] = block_centroids['geoid10']
node_pairs['starting_node'] = block_centroids['nearest_node']
node_pairs['ending_nodes'] = node_pairs['geoid10'].apply(lambda x: get_ending_nodes(x))

In [95]:
def get_shortest_walk(node_table_row, cur):
#cur is a cursor connected to an appropriate PostGIS database
    def compute_walking_times(starting_node, ending_nodes):
        array_string = f'ARRAY[{ending_nodes[0]}, {ending_nodes[1]}, {ending_nodes[2]}]'
        query = f'''SELECT * FROM pgr_bdDijkstra(
        'SELECT id, source, target, cost, reverse_cost FROM street_centerlines', {starting_node},  {array_string}, false);'''
        cur.execute(query)
        results = pd.DataFrame(cur.fetchall())
        cols = results.columns
        try:
            return results[results[cols[4]]==-1][cols[6]].tolist()
        except:
            return [0, 0, 0]
    
    return min(compute_walking_times(node_table_row['starting_node'], node_table_row['ending_nodes']))

In [96]:
conn = psycopg2.connect("dbname=FoodAccess user=kyleslugg")
cur = conn.cursor()

block_centroids['time_to_market'] = node_pairs.apply(lambda row: get_shortest_walk(row, cur), axis=1)

conn.close()
    

In [98]:
block_centroids['geom'] = block_centroids['geom'].apply(lambda x: WKTElement(x.wkt, srid=4326))
block_centroids.to_sql('block_centroids', engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid=4326)})